In [1]:
import pandas as pd
import numpy as np
import os
import csv
import arcpy
from scipy.interpolate import griddata
from arcpy import env

RuntimeError: The Product License has not been initialized.

# import well completion reports and AEM coarse fractions combined 
# we assume you have combined and orginized these data so far
# you aslo need to import IWFM conceptual model layers here we are using C2VSimFG model version 1.5( 4 layers)

In [ ]:
# Load the data
filtered_df = pd.read_csv("C:/Users/betebari/Documents/C2VSim_Texture/OSWCR/8-updated_all.csv")

# Create the new column for CLAY FRACTION
filtered_df['ClayInterbed'] = (100.0 - filtered_df['Coarse'] )
# or can be Descritption Column containing Clay  or Cl, CH, etc

# Calculate total records and number of unique values
total_records = len(filtered_df)

# Find number of unique values in the WCRNUMBER column
unique_wcrnumber = filtered_df['WellName'].nunique()

# Print the result
# Print total records and unique values per column
print(f"Total records: {total_records}")
print(f"Number of unique WCRNUMBER values: {unique_wcrnumber}")

filtered_df.head()

In [ ]:
C2VSimFG_df = pd.read_csv("C:/c2vsimfg1.5/C2VSimFG_node_layering_assigned.csv")

# Drop the individual columns if not needed anymore
C2VSimFG_df.drop(columns=['TotalThickness','top of BOFW','DC1','DC2','DC3','DC4','GSE'], inplace=True)

C2VSimFG_df.head()

In [ ]:
# Ensure the DataFrame contains X and Y columns
if 'X' not in filtered_df.columns or 'Y' not in filtered_df.columns:
    raise ValueError("The DataFrame must contain 'X' and 'Y' columns for spatial coordinates.")

# Export to CSV
csv_file = "Filtered_WCRs_AEM.csv"
filtered_df.to_csv(csv_file, index=False)
print(f"CSV file exported: {csv_file}")

In [ ]:
# Drop rows where 'ClayInterbed' is NaN and also drop rows where 'ClayInterbed' >= 80
filtered_df = filtered_df.dropna(subset=['ClayInterbed'])
filtered_df = filtered_df[filtered_df['ClayInterbed'] > 80]
filtered_df['ClaybedThickness'] = filtered_df[['INTERVALEND'] - filtered_df[['INTERVALSTART'] 

# Drop the individual columns if not needed anymore
filtered_df.drop(columns=['Kxy', 'SY','Ss','Kv','Coarse'], inplace=True)

# Calculate total records and number of unique values
total_records = len(filtered_df)

# Find number of unique values in the WCRNUMBER column
unique_wcrnumber = filtered_df['WellName'].nunique()

# Print total records and unique values per column
print(f"Total records: {total_records}")
print(f"Number of unique WCRNUMBER values: {unique_wcrnumber}")

# Print the result
filtered_df.head()

# Apply Interpoltion method on C2VSimFG Layer 1 through Layer 4

In [ ]:
# Set the workspace
env.workspace = "C:/Users/betebari/Documents/InSAR-Subsidence"
env.overwriteOutput = True

In [ ]:
gdb_path = "C:/Users/betebari/Documents/InSAR-Subsidence/Subsidence.gdb"
if not arcpy.Exists(gdb_path):
    arcpy.management.CreateFileGDB(os.path.dirname(gdb_path), os.path.basename(gdb_path))
    print(f"Geodatabase created: {gdb_path}")
else:
    print(f"Geodatabase already exists: {gdb_path}")

In [ ]:
# File paths
csv_file = "C:/Users/betebari/Documents/InSAR-Subsidence/Filtered_WCRs_AEM.csv"
gdb_path = "C:/Users/betebari/Documents/InSAR-Subsidence/Subsidence.gdb"
output_fc = f"{gdb_path}/Filtered_WCRs_AEM"

# Verify the CSV file exists
if not os.path.exists(csv_file):
    raise FileNotFoundError(f"CSV file not found: {csv_file}")
else:
    print(f"CSV file found: {csv_file}")

# Create the geodatabase if it doesn't exist
if not arcpy.Exists(gdb_path):
    arcpy.management.CreateFileGDB(os.path.dirname(gdb_path), os.path.basename(gdb_path))
    print(f"Created geodatabase: {gdb_path}")
else:
    print(f"Geodatabase exists: {gdb_path}")

# Delete existing feature class if necessary
if arcpy.Exists(output_fc):
    arcpy.management.Delete(output_fc)
    print(f"Deleted existing feature class: {output_fc}")

# Define spatial reference
spatial_ref = arcpy.SpatialReference(26910)  # UTM Zone 10N NAD83

# Create the point feature class
try:
    arcpy.management.XYTableToPoint(
        in_table=csv_file,
        out_feature_class=output_fc,
        x_field="X",  # Replace with actual X-coordinate column name
        y_field="Y",  # Replace with actual Y-coordinate column name
        coordinate_system=spatial_ref
    )
    print(f"Feature class created: {output_fc}")
except arcpy.ExecuteError:
    print(f"ArcPy ExecuteError: {arcpy.GetMessages(2)}")

In [ ]:
from arcpy.sa import Spline

# Set environment
arcpy.env.overwriteOutput = True
spatial_ref = arcpy.SpatialReference(26910)

# Check Spatial Analyst extension
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out")
else:
    raise RuntimeError("Spatial Analyst extension is required but not available.")

try:
    # Ensure layers exist
    layers = ['L1','A2','L2', 'L3', 'L4']
    coords_df = C2VSimFG_df.copy()

    # Check X and Y columns
    if 'X' not in coords_df.columns or 'Y' not in coords_df.columns:
        raise ValueError("The DataFrame must contain 'X' and 'Y' columns for spatial coordinates.")

    # Calculate cumulative values for layers
    coords_df['L1_cum'] = coords_df['L1']  # L1_cum = L1
    coords_df['L2_cum'] = coords_df['L1'] + coords_df['L2'] + coords_df['A2']  # L2_cum = L1 + A2 +L2
    coords_df['L3_cum'] = coords_df['L2_cum'] +  coords_df['L3']  # L3_cum = L1 + A2 + L2 + L3
    coords_df['L4_cum'] = coords_df['L3_cum']  + coords_df['L4']  # L4_cum = L1 + A2 + L2 + L3 + L4

    # Replace NaN or missing values with 0 if needed
    coords_df[['L1_cum', 'L2_cum', 'L3_cum', 'L4_cum']] = coords_df[['L1_cum', 'L2_cum', 'L3_cum', 'L4_cum']].fillna(0)

    # Export DataFrame to CSV
    csv_path = "Cumulative_Layers.csv"
    coords_df.to_csv(csv_path, index=False)
    print(f"CSV file with cumulative layers exported: {csv_path}")

    # Create point feature class
    gdb_path = "C:/Users/betebari/Documents/InSAR-Subsidence/Subsidence.gdb"
    point_fc = os.path.join(gdb_path, "point_fc")

    # Create geodatabase if it doesn't exist
    if not arcpy.Exists(gdb_path):
        arcpy.management.CreateFileGDB(os.path.dirname(gdb_path), os.path.basename(gdb_path))
        print(f"Geodatabase created: {gdb_path}")

    # Create point feature class from the cumulative CSV
    arcpy.management.XYTableToPoint(
        in_table=csv_path,
        out_feature_class=point_fc,
        x_field="X",
        y_field="Y",
        coordinate_system=spatial_ref
    )
    print(f"Feature class created: {point_fc}")

    # List fields in the feature class
    fields = [field.name for field in arcpy.ListFields(point_fc)]
    print("Fields in the feature class:", fields)

    # Perform spline interpolation for each cumulative layer
    cumulative_layers = ['L1_cum', 'L2_cum', 'L3_cum', 'L4_cum']
    output_dir = "C:/Users/betebari/Documents/InSAR-Subsidence"
    cell_size = 500  # Grid resolution

    for layer in cumulative_layers:
        output_raster = os.path.join(output_dir, f"{layer}_interpolated.tif")
        try:
            print(f"Running Spline interpolation for {layer}...")
            spline_result = Spline(point_fc, layer, cell_size)
            spline_result.save(output_raster)
            print(f"Spline interpolation complete for {layer}. Raster saved at {output_raster}")
        except arcpy.ExecuteError as e:
            print(f"ArcPy ExecuteError for {layer}: {e}")
        except Exception as e:
            print(f"Unexpected error for {layer}: {e}")

except arcpy.ExecuteError as e:
    print(f"ArcPy ExecuteError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    arcpy.CheckInExtension("Spatial")
    print("Spatial Analyst extension checked in")

# ExtractValuesToPoints operation(Interpolated C2VSimFG-L1 Thickness) to  points ((Filtered_WCRs_n_AEM ))

In [ ]:
# Step 1: Sanitize field names
filtered_df.columns = [col.replace(' ', '_').replace('-', '_')[:64] for col in filtered_df.columns]

# Step 2: Handle object fields
for col in filtered_df.select_dtypes(include=['object']).columns:
    filtered_df[col] = filtered_df[col].astype(str).str[:255]  # Convert to string and limit to 255 characters

# Step 3: Replace null values
filtered_df = filtered_df.fillna(0)  # Replace NaN with 0 for all columns

# Step 4: Convert to NumPy array
array = np.array(np.rec.fromrecords(filtered_df.values, names=filtered_df.columns.tolist()))

# Step 5: Define output geodatabase and feature class
gdb_path = "C:/Users/betebari/Documents/InSAR-Subsidence"
if not arcpy.Exists(gdb_path):
    arcpy.management.CreateFileGDB(os.path.dirname(gdb_path), os.path.basename(gdb_path))
    print(f"Created geodatabase: {gdb_path}")

output_fc = f"{gdb_path}/Filtered_WCRs_AEM"

# Step 6: Define spatial reference
spatial_ref = arcpy.SpatialReference(26910)  # UTM Zone 10N

# Step 7: Check if feature class exists, and delete it
if arcpy.Exists(output_fc):
    arcpy.management.Delete(output_fc)
    print(f"Deleted existing feature class: {output_fc}")

# Step 8: Ensure 'X' and 'Y' fields exist
if 'X' not in filtered_df.columns or 'Y' not in filtered_df.columns:
    raise ValueError("Columns 'X' and 'Y' are required but not found in the DataFrame.")

# Step 9: Convert NumPy array to feature class
arcpy.da.NumPyArrayToFeatureClass(array, output_fc, ['X', 'Y'], spatial_ref)

print(f"Feature class saved as: {output_fc}")

In [ ]:
# Path where the geodatabase will be created
workspace = "C:/Users/betebari/Documents/InSAR-Subsidence"
gdb_name = "Subsidence.gdb"

# Full path to the geodatabase
gdb_path = os.path.join(workspace, gdb_name)

# Check if the geodatabase already exists
if not arcpy.Exists(gdb_path):
    # Create the geodatabase
    arcpy.management.CreateFileGDB(workspace, gdb_name)
    print(f"Geodatabase created: {gdb_path}")
else:
    print(f"Geodatabase already exists: {gdb_path}")

In [ ]:
# Path to the CSV file (replace this with your actual file)
csv_file = "Filtered_WCRs_AEM.csv"

# Path to the geodatabase
gdb_path = "C:/Users/betebari/Documents/InSAR-Subsidence/Subsidence.gdb"

# Output feature class name
output_fc = f"{gdb_path}/Filtered_WCRs_AEM"

# Define spatial reference (UTM Zone 10N)
spatial_ref = arcpy.SpatialReference(26910)

# Create the point feature class from CSV
arcpy.management.XYTableToPoint(
    in_table=csv_file,
    out_feature_class=output_fc,
    x_field="X",  # Replace with the name of the X-coordinate field in your CSV
    y_field="Y",  # Replace with the name of the Y-coordinate field in your CSV
    coordinate_system=spatial_ref
)

print(f"Feature class created: {output_fc}")

In [ ]:
# Check out the Spatial Analyst extension
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out")
else:
    raise RuntimeError("Spatial Analyst extension is not available or licensed.")

try:
    # Workspace and file paths
    arcpy.env.workspace = "C:/Users/betebari/Documents/InSAR-Subsidence/Subsidence1.gdb"
    raster_folder = "C:/Users/betebari/Documents/InSAR-Subsidence/"
    input_points = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb/Filtered_WCRs_AEM"

    # Validate input points feature class
    if not arcpy.Exists(input_points):
        raise FileNotFoundError(f"Input points feature class '{input_points}' does not exist.")

    # Check and remove the RASTERVALU field if it exists
    fields = [f.name for f in arcpy.ListFields(input_points)]
    if "RASTERVALU" in fields:
        print("Field 'RASTERVALU' exists. Deleting it to avoid conflicts...")
        arcpy.management.DeleteField(input_points, "RASTERVALU")
        print("Field 'RASTERVALU' deleted.")

    # List of rasters to process
    rasters = ['L1_cum_interpolated.tif', 'L2_cum_interpolated.tif', 'L3_cum_interpolated.tif', 'L4_cum_interpolated.tif']

    # Iterate over raster files
    for raster in rasters:
        print(f"Processing {raster}...")

        # Define raster and output feature class
        raster_path = os.path.join(raster_folder, raster)
        if not arcpy.Exists(raster_path):
            print(f"Raster '{raster_path}' does not exist. Skipping...")
            continue

        output_fc = os.path.join(arcpy.env.workspace, f"Extracted_{os.path.splitext(raster)[0]}")

        # Perform ExtractValuesToPoints
        arcpy.sa.ExtractValuesToPoints(
            in_point_features=input_points,
            in_raster=raster_path,
            out_point_features=output_fc,
            interpolate_values="NONE",
            add_attributes="VALUE_ONLY"
        )
        print(f"Values extracted to {output_fc}")

        # Export to CSV
        csv_output = os.path.join(raster_folder, f"{os.path.splitext(raster)[0]}_output.csv")
        fields = [f.name for f in arcpy.ListFields(output_fc) if f.name.lower() != 'shape']
        fields.append('SHAPE@XY')

        with open(csv_output, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(fields[:-1] + ['X', 'Y'])

            with arcpy.da.SearchCursor(output_fc, fields) as cursor:
                for row in cursor:
                    row_list = list(row[:-1])
                    x, y = row[-1]
                    row_list.extend([x, y])
                    writer.writerow(row_list)

        print(f"CSV exported to {csv_output}")

except arcpy.ExecuteError as e:
    print(f"ArcPy ExecuteError: {e}")
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    arcpy.CheckInExtension("Spatial")
    print("Spatial Analyst extension checked in")